In [ ]:
!pip install tvm_valuetypes

In [1]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM
import codecs
from tvm_valuetypes import Cell

# Easy test examples

In [2]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

DEBUG: Start parse code
DEBUG: Start parse base64
DEBUG: Start parse boc
DEBUG: Code parsed success
Code loaded: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Use code: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Load cp0
[ 4][t 0][2022-10-19 22:53:33.974571][arithops.cpp:34]	execute PUSHINT 1

[ 4][t 0][2022-10-19 22:53:33.974922][vm.cpp:450]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0
Got 1 from TVM: 1


In [3]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()

cell = Cell()
b64 = tvm_result[0]
cell.data.from_bytes(codecs.decode(codecs.encode(b64, 'utf8'), 'base64'))

cell.data # This is continuation

b'\xb5\xee\x9cr\xe1\x01\x02\x01\x00\x0e\x00\x12\x1c\x01\x0c\x04\x00\x00\x00\x06\x00\x01\x00\x06q\xedC\x07\xb93\xa0'

In [4]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

[124711402,
 0,
 0,
 0,
 0,
 0,
 0,
 [100000, None],
 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM=',
 None]

## Change c7

In [5]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [6]:
tvm.set_c7(unixtime=179, 
           randseed=100, 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=100000000)

In [7]:
tvm.run_vm()

[[124711402,
  0,
  0,
  179,
  0,
  0,
  100,
  [100000000, None],
  'te6ccuEBAQEAJABIAEOADVySWN59rHc58b7yQKDOcZsaNW4TYZQKwLQsP8G0qTtQ5qaveg==',
  None]]

## Get detailed result

In [16]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,)

Exit code:  -1 
C5:  te6ccuEBAQEAAgAEAABmLc6k 
Gas credit:  0 
Gas used:  31 
C4:  te6ccuEBAQEAAgAEAABmLc6k 
VM Steps:  2 
VM final hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM init hash:  0000000000000000000000000000000000000000000000000000000000000000
